In [1]:
import io
arquivo = io.open('artigos.txt', 'r', encoding='utf-8')
artigos = arquivo.read()
print(f'O arquivo "artigos.txt" foi carregado e possui {len(artigos)} caracteres.')

O arquivo "artigos.txt" foi carregado e possui 2605046 caracteres.


In [2]:
import nltk
nltk.download('punkt')

def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print(f'O arquivo "artigos.txt" possui {len(lista_palavras)} palavras.')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaoteixeira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


O arquivo "artigos.txt" possui 403031 palavras.


In [3]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
palavras_unicas = set(lista_normalizada)
print(f'O arquivo "artigos.txt" possui {len(palavras_unicas)} palavras únicas.')
print(f'Exemplo: {lista_normalizada[:10]} ...')

O arquivo "artigos.txt" possui 18464 palavras únicas.
Exemplo: ['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no'] ...


In [4]:
frequencia = nltk.FreqDist(lista_normalizada)
print(f'As 10 palavras que mais ocorrem são: {frequencia.most_common(10)}')

As 10 palavras que mais ocorrem são: [('de', 15502), ('o', 14056), ('que', 12230), ('a', 11099), ('e', 10501), ('para', 7710), ('um', 6367), ('é', 5899), ('uma', 5220), ('do', 5124)]


In [5]:
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding='utf-8')
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

lista_teste = cria_dados_teste('palavras.txt')
print(f'O arquivo "palavras.txt" foi carregado e possui {len(lista_teste)} pares de palavras para testes de eficiência do corretor.')
print(f'Exemplo: {lista_teste[:5]} ...')

O arquivo "palavras.txt" foi carregado e possui 186 pares de palavras para testes de eficiência do corretor.
Exemplo: [('podemos', 'pyodemos'), ('esse', 'esje'), ('já', 'jrá'), ('nosso', 'nossov'), ('são', 'sãêo')] ...


In [6]:
total_palavras = len(lista_normalizada)
letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'

def insere_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

def deleta_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

def troca_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

def inverte_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

def gerador_palavras_1(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deleta_letras(fatias)
    palavras_geradas += troca_letras(fatias)
    palavras_geradas += inverte_letras(fatias)
    return palavras_geradas

def gerador_palavras_2(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras_1(palavra)
    return novas_palavras

def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

def corretor_dist1(palavra):
    palavras_geradas = gerador_palavras_1(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

def corretor_dist2(palavra):
    palavras_geradas_1 = gerador_palavras_1(palavra)
    palavras_geradas_2 = gerador_palavras_2(palavras_geradas_1)
    todas_palavras = set(palavras_geradas_1 + palavras_geradas_2)
    candidatos = [palavra]
    for p in todas_palavras:
        if p in palavras_unicas:
            candidatos.append(p)
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

def avaliador(testes):
    num_palavras = len(testes)
    acertou = 0
    desconhecidas = 0
    for correta, errada in testes:
        palavra_corrigida = corretor_dist2(errada)
        if correta not in palavras_unicas:
            desconhecidas +=1
            palavras_unicas.add(correta)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            palavra_corrigida = corretor_dist1(errada)
            if palavra_corrigida  == correta:
                acertou += 1
    taxa_palavras_desconhecidas = round((desconhecidas/num_palavras)*100,2) 
    taxa_acerto = round((acertou/num_palavras)*100,2)
    print(f'A taxa de acerto é: {taxa_acerto}%')
    print(f'A taxa de palavras desconhecidas é: {taxa_palavras_desconhecidas}%')

In [7]:
avaliador(lista_teste)

A taxa de acerto é: 85.48%
A taxa de palavras desconhecidas é: 5.38%


In [8]:
palavra_teste = 'lóigca'    # os dois coretores acertam
print(corretor_dist1(palavra_teste))
print(corretor_dist2(palavra_teste))

lógica
lógica


In [9]:
palavra_teste = 'lóigcca'   # apenas o corretor 2 acerta
print(corretor_dist1(palavra_teste))
print(corretor_dist2(palavra_teste))

alóigcca
lógica


In [10]:
palavra_teste = 'esze'      # apenas o corretor 1 acerta
print(corretor_dist1(palavra_teste))
print(corretor_dist2(palavra_teste))

esse
se


In [11]:
palavra_teste = 'deset'     # apenas o corretor 1 acerta
print(corretor_dist1(palavra_teste))
print(corretor_dist2(palavra_teste))

deste
dessa
